In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, trim, lower, regexp_replace, mean, stddev
from pyspark.sql.types import IntegerType, DoubleType, StringType

In [0]:
# Create Spark session
spark = SparkSession.builder.appName("ChurnDataCleaning").getOrCreate()

In [0]:
catalog_name = dbutils.widgets.get('catalog')
schema_raw = dbutils.widgets.get('schema_raw')
shema_datamart = dbutils.widgets.get('shema_datamart')

table_inter_hf = 'raw_customer_churn_hf_temp'
table_inter_sf = 'raw_customer_churn_sf_temp'

In [0]:
df1 = spark.table(f'{catalog_name}.{schema_raw}.{table_inter_hf}')
df2 = spark.table(f'{catalog_name}.{schema_raw}.{table_inter_sf}')

In [0]:
feature_cols = ['CustomerID','Age','Gender','Tenure','MonthlyCharges','TotalCharges','ContractType','PaymentMethod','Churn']

df1 = df1.select(feature_cols)
df2 = df2.select(feature_cols)

In [0]:
df = df1.union(df2)

In [0]:
df = df.dropDuplicates()

In [0]:
string_cols = [f.name for f in df.schema.fields if isinstance(f.dataType, StringType)]

# Lowercase, trim spaces for all string columns
for col_name in string_cols:
    df = df.withColumn(col_name, trim(lower(col(col_name))))

In [0]:
# Fix known categorical inconsistencies
df = df.withColumn("Gender", 
                   when(col("Gender").isin("male", "m"), "male")
                   .when(col("Gender").isin("female", "f"), "female")
                   .otherwise(col("Gender")))

In [0]:
df = df.withColumn("Internet Type", regexp_replace(col("Internet Type"), "fiber optic", "fiber optic"))

In [0]:
# Fill numeric columns with median, categorical with mode
numeric_cols = [f.name for f in df.schema.fields if isinstance(f.dataType, (IntegerType, DoubleType))]
categorical_cols = [c for c in df.columns if c not in numeric_cols]

# Numeric: Fill with median
for col_name in numeric_cols:
    median_val = df.approxQuantile(col_name, [0.5], 0.01)[0]
    df = df.na.fill({col_name: median_val})

# Categorical: Fill with mode
for col_name in categorical_cols:
    mode_val = df.groupBy(col_name).count().orderBy(col("count").desc()).first()[0]
    df = df.na.fill({col_name: mode_val})

In [0]:
binary_cols = ["Married", "Paperless Billing", "Online Security", 
               "Device Protection Plan", "Streaming TV", 
               "Streaming Movies", "Unlimited Data", "Churn"]

for col_name in binary_cols:
    df = df.withColumn(col_name, col(col_name).cast(IntegerType()))

df = df.withColumn("Monthly Charge", col("Monthly Charge").cast(DoubleType()))
df = df.withColumn("Total Charges", col("Total Charges").cast(DoubleType()))
df = df.withColumn("Satisfaction Score", col("Satisfaction Score").cast(IntegerType()))
df = df.withColumn("Number of Dependents", col("Number of Dependents").cast(IntegerType()))


In [0]:
for col_name in ["Monthly Charge", "Total Charges", "Age", "Tenure in Months"]:
    stats = df.select(mean(col_name).alias("mean"), stddev(col_name).alias("std")).collect()[0]
    mean_val, std_val = stats["mean"], stats["std"]
    df = df.filter(((col(col_name) - mean_val) / std_val).abs() < 3)


In [0]:
# Senior Citizen flag
df = df.withColumn("Senior Citizen", when(col("Age") >= 60, 1).otherwise(0))

# Flag for long tenure customers
df = df.withColumn("Long Tenure Flag", when(col("Tenure in Months") >= 24, 1).otherwise(0))

In [0]:
df.printSchema()
df.show(10, truncate=False)

In [0]:
# sample data quality report in pdf / csv format . summarising issues and resolution